In [3]:
import pandas as pd

# Step 1: Load the CSV file
df = pd.read_csv('path_to_file.csv')

# Function to clean currency values and convert them to numeric
def clean_currency(value):
    # Remove dollar sign and commas, then convert to float
    if isinstance(value, str):
        value = value.replace('$', '').replace(',', '')
    try:
        return float(value)
    except ValueError:
        return None  # Return None if conversion fails

# Step 2: Apply the cleaning function to relevant columns
df['Total Revenue'] = df['Total Revenue'].apply(clean_currency)
df['Net Income'] = df['Net Income'].apply(clean_currency)
df['Total Assets'] = df['Total Assets'].apply(clean_currency)
df['Total Liabilities'] = df['Total Liabilities'].apply(clean_currency)
df['Cash Flow from Operating Activities'] = df['Cash Flow from Operating Activities'].apply(clean_currency)

# Display the cleaned data
print("After cleaning the currency values:")
print(df.head())

# Step 3: Handle missing values by estimating growth rates (we'll do this for Total Revenue, Net Income, Total Assets, etc.)
def estimate_growth(previous_value, current_value):
    if previous_value is not None and current_value is not None:
        growth_rate = (current_value - previous_value) / previous_value
        return previous_value * (1 + growth_rate)
    return None

# Iterate over each row and estimate missing values
for index, row in df.iterrows():
    if pd.isnull(row['Total Revenue']) and index > 0:
        df.loc[index, 'Total Revenue'] = estimate_growth(df.loc[index - 1, 'Total Revenue'], df.loc[index - 1, 'Total Revenue'])
        
    if pd.isnull(row['Net Income']) and index > 0:
        df.loc[index, 'Net Income'] = estimate_growth(df.loc[index - 1, 'Net Income'], df.loc[index - 1, 'Net Income'])

    if pd.isnull(row['Total Assets']) and index > 0:
        df.loc[index, 'Total Assets'] = estimate_growth(df.loc[index - 1, 'Total Assets'], df.loc[index - 1, 'Total Assets'])

    if pd.isnull(row['Total Liabilities']) and index > 0:
        df.loc[index, 'Total Liabilities'] = estimate_growth(df.loc[index - 1, 'Total Liabilities'], df.loc[index - 1, 'Total Liabilities'])

    if pd.isnull(row['Cash Flow from Operating Activities']) and index > 0:
        df.loc[index, 'Cash Flow from Operating Activities'] = estimate_growth(df.loc[index - 1, 'Cash Flow from Operating Activities'], df.loc[index - 1, 'Cash Flow from Operating Activities'])

# Display the DataFrame after filling missing values
print("After filling missing values:")
print(df)

# Step 4: Now, you can proceed with analysis, for example, calculating year-over-year growth rates or generating other metrics.

# Example of calculating growth rates:
df['Revenue Growth (%)'] = df.groupby('Company')['Total Revenue'].pct_change() * 100
df['Net Income Growth (%)'] = df.groupby('Company')['Net Income'].pct_change() * 100

# Display the data with the calculated growth rates
print(df)

# Step 5: Export the cleaned DataFrame to a new CSV (optional)
df.to_csv('cleaned_financial_data.csv', index=False)


After cleaning the currency values:
     Company  Year  Total Revenue  Net Income  Total Assets  \
0  Microsoft  2024       245122.0     88136.0      512163.0   
1  Microsoft  2023       211915.0     72361.0      411976.0   
2  Microsoft  2022       198270.0     72738.0           NaN   
3      Tesla  2024            NaN         NaN           NaN   
4      Tesla  2023        96773.0     14974.0      106618.0   

   Total Liabilities  Cash Flow from Operating Activities  
0           243686.0                             118548.0  
1           205753.0                              87582.0  
2                NaN                              89035.0  
3                NaN                                  NaN  
4            43009.0                              13256.0  
After filling missing values:
     Company  Year  Total Revenue  Net Income  Total Assets  \
0  Microsoft  2024       245122.0     88136.0      512163.0   
1  Microsoft  2023       211915.0     72361.0      411976.0   
2  Mic